# 범주형 데이터 다루기

범주형 데이터 데이터 인코딩 기능

* 레이블 인코딩(label encoding) : 말 그대로 레이블링을 한다. ex) 자동차 : 1, TV : 2 
* 원-핫 인코딩(one hot encoding) : 원핫은 [0,0,1], [0, 1, 0], [1, 0, 0]과 같이 벡터 안에 하나의 값만 1이고 나머지는 0으로 표현하는 방법

## Pandas 범주형 데이터 처리하기

### 샘플데이터 


In [36]:
df = pd.DataFrame({"id":[1,2,3,4,5,6], "raw_grade":['A', 'B', 'B', 'A', 'A', 'F']})
df

,id,raw_grade
0,1,A
1,2,B
2,3,B
3,4,A
4,5,A
5,6,F


### 범주형 데이터로 변환
가공하지 않은 성적을 범주형 데이터로 변환합니다.

In [37]:
df["grade"] = df["raw_grade"].astype("category")
df

,id,raw_grade,grade
0,1,A,A
1,2,B,B
2,3,B,B
3,4,A,A
4,5,A,A
5,6,F,F


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   id         6 non-null      int64   
 1   raw_grade  6 non-null      object  
 2   grade      6 non-null      category
dtypes: category(1), int64(1), object(1)
memory usage: 334.0+ bytes


### 범주 확인 및 범주의 이름 변경
범주에 더 의미 있는 이름을 붙일 수 있다.   
Series.cat.categories로 할당하는 것이 적합하다.

In [39]:
df["grade"].cat.categories

Index(['A', 'B', 'F'], dtype='object')

In [33]:
df["grade"].cat.categories = ["very good", "good", "very bad"]
df

,id,raw_grade,grade
0,1,A,very good
1,2,B,good
2,3,B,good
3,4,A,very good
4,5,A,very good
5,6,F,very bad


In [32]:
df["grade"]

0    very good
1         good
2         good
3    very good
4    very good
5     very bad
Name: grade, dtype: category
Categories (3, object): [very good, good, very bad]

### 새로운 범주의 설정
범주의 순서를 바꾸고 동시에 누락된 범주를 추가한다.  
cat.set_categories() 함수에 리스트 형식으로 인자를 넣어줘야 한다.  
Series.cat에 속하는 메소드는 기본적으로 새로운 Series를 리턴한다.

In [24]:
df["grade"] = df["grade"].cat.set_categories(["very bad", "bad", "medium", "good", "very good"])

df

,id,raw_grade,grade
0,1,A,very good
1,2,B,good
2,3,B,good
3,4,A,very good
4,5,A,very good
5,6,F,very bad


### 범주형 데이터의 정렬
정렬은 사전 순서(ABC순서)가 아닌, 해당 범주에서 지정된 순서대로 배열된다.

very bad, bad, medium, good, very good 의 순서로 기재되어 있기 때문에 정렬 결과도 해당 순서대로 배열된다.

In [25]:
df.sort_values(by="grade")

,id,raw_grade,grade
5,6,F,very bad
1,2,B,good
2,3,B,good
0,1,A,very good
3,4,A,very good
4,5,A,very good


### 범주형 데이터의 그룹화
범주의 열을 기준으로 그룹화하면 빈 범주도 표시된다.

In [26]:
df.groupby("grade").size()

grade
very bad     1
bad          0
medium       0
good         2
very good    3
dtype: int64

## 순서가 있는 특성과 순서가 없는 특성

In [13]:
import pandas as pd

df = pd.DataFrame([['green', 'M', 10.1, 'class1'],
                   ['red', 'L', 13.5, 'class2'],
                   ['blue', 'XL', 15.3, 'class1']])

df.columns = ['color', 'size', 'price', 'classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


## 순서 특성 매핑하기

In [14]:
size_mapping = {'XL': 3,
                'L': 2,
                'M': 1}

df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


In [15]:
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

## 클래스 레이블 인코딩

In [16]:
import numpy as np

# 문자열 클래스 레이블을 정수로 변환하기 위해
# 매핑 딕셔너리를 만듭니다.
class_mapping = {label: idx for idx, label in enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

In [17]:
# 문자열 클래스 레이블을 정수로 변환합니다.
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


In [18]:
# 클래스 레이블 매핑을 거꾸로 수행합니다.
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class1
1,red,2,13.5,class2
2,blue,3,15.3,class1


### 사이킷런의 LabelEncoder를 사용하기

In [19]:
from sklearn.preprocessing import LabelEncoder

# 사이킷런의 LabelEncoder를 사용하여 레이블을 인코딩합니다.
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

array([0, 1, 0])

In [20]:
# 거꾸로 매핑합니다.
class_le.inverse_transform(y)

array(['class1', 'class2', 'class1'], dtype=object)

### 판다스 원-핫 인코딩 사용하기

In [21]:
# 판다스의 원-핫 인코딩

pd.get_dummies(df[['price', 'color', 'size']])

,price,size,color_blue,color_green,color_red
0,10.1,1,0,1,0
1,13.5,2,0,0,1
2,15.3,3,1,0,0


In [22]:
pd.get_dummies(df[['price', 'color', 'size']], columns=['size'])

,price,color,size_1,size_2,size_3
0,10.1,green,1,0,0
1,13.5,red,0,1,0
2,15.3,blue,0,0,1


In [23]:
pd.get_dummies(df[['price', 'color', 'size']], drop_first=True)

,price,size,color_green,color_red
0,10.1,1,1,0
1,13.5,2,0,1
2,15.3,3,0,0


### 사이킷런의 원-핫 인코딩
사이킷런의 OneHotEncoder 사용

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

items = ['TV','냉장고','세탁기','냉장고','TV']

In [2]:
# label encoder
encoder = LabelEncoder()
encoder.fit(items)
labels = encoder.transform(items)
labels = labels.reshape(-1,1)
print(labels)

[[0]
 [1]
 [2]
 [1]
 [0]]


In [4]:
# one-hot encoder
one_hot = OneHotEncoder()
one_hot.fit(labels)
onehot_labels = one_hot.transform(labels)

print(onehot_labels.toarray())

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]]
